In [2]:
import spacy
import random
from collections import defaultdict

# Load NLP model for entity recognition
nlp = spacy.load("en_core_web_sm")

# Sample database of community resources with eligibility criteria
resources = {
    "healthcare": [
        {"name": "Free Clinic", "eligibility": ["low income", "uninsured"]},
        {"name": "Community Health Center", "eligibility": ["general public"]},
        {"name": "Mental Health Hotline", "eligibility": ["anyone"]},
    ],
    "housing": [
        {"name": "Emergency Shelter", "eligibility": ["homeless"]},
        {"name": "Affordable Housing Program", "eligibility": ["low income"]},
        {"name": "Rent Assistance", "eligibility": ["families", "low income"]},
    ],
    "legal aid": [
        {"name": "Pro Bono Legal Services", "eligibility": ["low income"]},
        {"name": "Tenant Rights Advocacy", "eligibility": ["renters"]},
        {"name": "Immigration Assistance", "eligibility": ["immigrants"]},
    ],
    "food assistance": [
        {"name": "Food Bank", "eligibility": ["anyone"]},
        {"name": "SNAP Enrollment Help", "eligibility": ["low income"]},
        {"name": "Community Kitchen", "eligibility": ["anyone"]},
    ]
}

# Simulated fairness check for urban vs. rural access
demographic_data = {
    "urban": ["Community Health Center", "Pro Bono Legal Services", "Food Bank"],
    "rural": ["Mental Health Hotline", "Tenant Rights Advocacy", "Community Kitchen"]
}

# Logging system for evaluation and feedback
referral_log = []
user_feedback = []

def check_bias(referrals, user_location):
    """Check for potential biases in recommendations based on location."""
    flagged = [r["name"] for r in referrals if r["name"] in demographic_data["urban"] and user_location == "rural"]
    if flagged:
        print("⚠️ Potential bias detected: Some recommendations may not be optimized for rural users.")
    return flagged

def match_resources(user_input, user_location="urban", user_profile=None):
    """Processes user request and returns recommended resources."""
    doc = nlp(user_input.lower())

    matched_categories = set()
    for token in doc:
        if token.lemma_ in resources:
            matched_categories.add(token.lemma_)

    if not matched_categories:
        return "Sorry, I couldn't find a matching resource category. Could you rephrase your request?"

    referrals = []
    for category in matched_categories:
        for resource in resources[category]:
            # Check if user meets eligibility criteria (if provided)
            if user_profile and any(criterion in user_profile for criterion in resource["eligibility"]):
                referrals.append(resource)
            elif not resource["eligibility"]:  # No restrictions
                referrals.append(resource)

    # If no eligibility-matching results, provide general results
    if not referrals:
        referrals = [r for cat in matched_categories for r in resources[cat]]

    # Bias check
    flagged_referrals = check_bias(referrals, user_location)

    # Log the referral for evaluation
    referral_log.append({
        "user_input": user_input,
        "location": user_location,
        "profile": user_profile,
        "referrals": referrals,
        "flagged": flagged_referrals
    })

    return f"Recommended resources: {', '.join(r['name'] for r in referrals)}"

def collect_feedback(user_input, rating, comments):
    """Allows users to provide feedback on AI referrals."""
    user_feedback.append({"query": user_input, "rating": rating, "comments": comments})
    print("Thank you for your feedback! Your input helps improve the system.")

# Example interaction
user_request = "I need legal aid for housing issues."
user_location = "rural"
user_profile = ["renters", "low income"]  # User characteristics for eligibility

response = match_resources(user_request, user_location, user_profile)
print(response)

# Example feedback collection
collect_feedback(user_request, rating=4, comments="Helpful, but more options would be great.")


Recommended resources: Affordable Housing Program, Rent Assistance
Thank you for your feedback! Your input helps improve the system.


In [4]:
import spacy
import random
from collections import defaultdict

# Load NLP model for better entity recognition
nlp = spacy.load("en_core_web_sm")

# Sample structured database of resources with added urgency, eligibility, and access details
resources = {
    "healthcare": [
        {"name": "Free Clinic", "eligibility": ["low income", "uninsured"], "urgency": "medium", "location": "both"},
        {"name": "Community Health Center", "eligibility": ["general public"], "urgency": "low", "location": "urban"},
        {"name": "Mental Health Hotline", "eligibility": ["anyone"], "urgency": "high", "location": "both"},
    ],
    "housing": [
        {"name": "Emergency Shelter", "eligibility": ["homeless"], "urgency": "high", "location": "both"},
        {"name": "Affordable Housing Program", "eligibility": ["low income"], "urgency": "medium", "location": "urban"},
        {"name": "Rent Assistance", "eligibility": ["families", "low income"], "urgency": "medium", "location": "both"},
    ],
    "legal aid": [
        {"name": "Pro Bono Legal Services", "eligibility": ["low income"], "urgency": "medium", "location": "urban"},
        {"name": "Tenant Rights Advocacy", "eligibility": ["renters"], "urgency": "medium", "location": "both"},
        {"name": "Immigration Assistance", "eligibility": ["immigrants"], "urgency": "high", "location": "both"},
    ],
    "food assistance": [
        {"name": "Food Bank", "eligibility": ["anyone"], "urgency": "medium", "location": "both"},
        {"name": "SNAP Enrollment Help", "eligibility": ["low income"], "urgency": "low", "location": "both"},
        {"name": "Community Kitchen", "eligibility": ["anyone"], "urgency": "high", "location": "both"},
    ]
}

# Fairness check for rural vs. urban users
demographic_bias_check = {
    "urban": ["Community Health Center", "Pro Bono Legal Services", "Food Bank"],
    "rural": ["Mental Health Hotline", "Tenant Rights Advocacy", "Community Kitchen"]
}

# Logging system for transparency, evaluation, and improvement
referral_log = []
user_feedback = []

def detect_bias(recommendations, user_location):
    """Checks for location-based bias and flags urban-centric recommendations for rural users."""
    flagged = [r["name"] for r in recommendations if r["name"] in demographic_bias_check["urban"] and user_location == "rural"]
    if flagged:
        print("⚠️ Bias Alert: Some recommendations may not be fully optimized for rural users.")
    return flagged

def match_resources(user_input, user_location="urban", user_profile=None, urgency_level="medium"):
    """Processes user request, considers eligibility, urgency, and location bias, and returns relevant resources."""
    doc = nlp(user_input.lower())

    matched_categories = set()
    for token in doc:
        if token.lemma_ in resources:
            matched_categories.add(token.lemma_)

    if not matched_categories:
        return "Sorry, I couldn't find a matching resource category. Could you rephrase your request?"

    referrals = []
    for category in matched_categories:
        for resource in resources[category]:
            # Check eligibility
            if user_profile and any(criterion in user_profile for criterion in resource["eligibility"]):
                referrals.append(resource)
            elif not resource["eligibility"]:  # No restrictions
                referrals.append(resource)

    # If no matches found via eligibility, provide general category-based results
    if not referrals:
        referrals = [r for cat in matched_categories for r in resources[cat]]

    # Filter based on urgency (prioritize high urgency when needed)
    prioritized_referrals = sorted(referrals, key=lambda x: ["low", "medium", "high"].index(x["urgency"]), reverse=True)
    
    # Apply bias detection and log flagged recommendations
    flagged_referrals = detect_bias(prioritized_referrals, user_location)

    # Log referral details for auditing
    referral_log.append({
        "user_input": user_input,
        "location": user_location,
        "profile": user_profile,
        "urgency": urgency_level,
        "referrals": prioritized_referrals,
        "flagged": flagged_referrals
    })

    # Explanation of recommendations
    response = "Recommended resources:\n"
    for r in prioritized_referrals:
        response += f"- {r['name']} (Urgency: {r['urgency']}, Eligibility: {', '.join(r['eligibility'])})\n"

    return response

def collect_feedback(user_input, rating, comments):
    """Captures user feedback for continuous system improvement."""
    user_feedback.append({"query": user_input, "rating": rating, "comments": comments})
    print("Thank you for your feedback! Your input helps improve the system.")

# --- Example Usage ---

user_request = "I need legal aid for housing issues."
user_location = "rural"  # User is in a rural area
user_profile = ["renters", "low income"]  # User characteristics for eligibility
urgency_level = "high"  # Request is urgent

response = match_resources(user_request, user_location, user_profile, urgency_level)
print(response)

# Example feedback collection
collect_feedback(user_request, rating=4, comments="The recommendations were useful, but I need more details.")


Recommended resources:
- Affordable Housing Program (Urgency: medium, Eligibility: low income)
- Rent Assistance (Urgency: medium, Eligibility: families, low income)

Thank you for your feedback! Your input helps improve the system.


In [13]:
import spacy
import random

# Load NLP model for better entity recognition
nlp = spacy.load("en_core_web_sm")

# Structured database of resources with urgency, eligibility, and location-based bias handling
resources = {
    "healthcare": [
        {"name": "Free Clinic", "eligibility": ["low income", "uninsured"], "urgency": "medium", "location": "both"},
        {"name": "Community Health Center", "eligibility": ["general public"], "urgency": "low", "location": "urban"},
        {"name": "Mental Health Hotline", "eligibility": ["anyone"], "urgency": "high", "location": "both"},
    ],
    "housing": [
        {"name": "Emergency Shelter", "eligibility": ["homeless"], "urgency": "high", "location": "both"},
        {"name": "Affordable Housing Program", "eligibility": ["low income"], "urgency": "medium", "location": "urban"},
        {"name": "Rent Assistance", "eligibility": ["families", "low income"], "urgency": "medium", "location": "both"},
    ],
    "legal aid": [
        {"name": "Pro Bono Legal Services", "eligibility": ["low income"], "urgency": "medium", "location": "urban"},
        {"name": "Tenant Rights Advocacy", "eligibility": ["renters"], "urgency": "medium", "location": "both"},
        {"name": "Immigration Assistance", "eligibility": ["immigrants"], "urgency": "high", "location": "both"},
    ],
    "food assistance": [
        {"name": "Food Bank", "eligibility": ["anyone"], "urgency": "medium", "location": "both"},
        {"name": "SNAP Enrollment Help", "eligibility": ["low income"], "urgency": "low", "location": "both"},
        {"name": "Community Kitchen", "eligibility": ["anyone"], "urgency": "high", "location": "both"},
    ]
}

# Bias detection rules
bias_check = {
    "urban": ["Community Health Center", "Pro Bono Legal Services"],
    "rural": ["Mental Health Hotline", "Tenant Rights Advocacy"]
}

# System logging and user feedback tracking
referral_log = []
user_feedback = []

def detect_bias(recommendations, user_location):
    """Identifies location bias in recommendations and adjusts fairness dynamically."""
    flagged = [r["name"] for r in recommendations if r["name"] in bias_check["urban"] and user_location == "rural"]
    
    if flagged:
        print("⚠️ Bias Alert: Some recommendations may not fully consider rural accessibility. Adjusting suggestions...")

        # Adjust recommendations dynamically for fairness
        fair_recommendations = [r for r in recommendations if r["name"] not in flagged]
        if fair_recommendations:
            return fair_recommendations
    return recommendations

def match_resources(user_input, user_location="urban", user_profile=None, urgency_level="medium"):
    """Processes user request, considers eligibility, urgency, bias mitigation, and provides optimal referrals."""
    doc = nlp(user_input.lower())

    # Identify relevant categories using NLP
    matched_categories = {token.lemma_ for token in doc if token.lemma_ in resources}

    if not matched_categories:
        return "Sorry, I couldn't find a relevant resource. Could you provide more details?"

    referrals = []
    for category in matched_categories:
        for resource in resources[category]:
            if user_profile and any(criterion in user_profile for criterion in resource["eligibility"]):
                referrals.append(resource)
            elif not resource["eligibility"]:
                referrals.append(resource)

    if not referrals:
        referrals = [r for cat in matched_categories for r in resources[cat]]

    # Prioritize based on urgency
    prioritized_referrals = sorted(referrals, key=lambda x: ["low", "medium", "high"].index(x["urgency"]), reverse=True)

    # Apply bias correction
    unbiased_referrals = detect_bias(prioritized_referrals, user_location)

    # Log the recommendation for auditing
    referral_log.append({
        "user_input": user_input,
        "location": user_location,
        "profile": user_profile,
        "urgency": urgency_level,
        "referrals": unbiased_referrals
    })

    # Generate response with explainability
    response = "Recommended resources:\n"
    for r in unbiased_referrals:
        response += f"- {r['name']} (Urgency: {r['urgency']}, Eligibility: {', '.join(r['eligibility'])})\n"

    return response

def collect_feedback(user_input, rating, comments):
    """Captures user feedback for continuous improvement."""
    user_feedback.append({"query": user_input, "rating": rating, "comments": comments})
    print("Thank you for your feedback! Your input helps improve the system.")

# --- Example Usage ---

user_request = "I need legal aid for housing issues."
user_location = "rural"  # User is in a rural area
user_profile = ["renters", "low income"]  # User eligibility criteria
urgency_level = "high"  # Request is urgent

response = match_resources(user_request, user_location, user_profile, urgency_level)
print(response)

# Example feedback collection
collect_feedback(user_request, rating=4, comments="The recommendations were useful, but more rural options needed.")


Recommended resources:
- Affordable Housing Program (Urgency: medium, Eligibility: low income)
- Rent Assistance (Urgency: medium, Eligibility: families, low income)

Thank you for your feedback! Your input helps improve the system.


In [11]:
import transformers
print(transformers.__version__)


4.50.3


In [2]:
import spacy
from transformers import pipeline, GPTNeoForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline

# Load spaCy model for entity recognition or text preprocessing
nlp = spacy.load("en_core_web_sm")

# Load the model and tokenizer for GPT-Neo
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)

# Create a Hugging Face text generation pipeline with increased max_length
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, 
                     max_length=500, truncation=True)

# Wrap the pipeline with HuggingFacePipeline for LangChain compatibility
llm = HuggingFacePipeline(pipeline=generator)

# Function to process the question using spaCy before passing it to the model
def process_and_ask(question):
    # Process text using spaCy for better understanding of the entities in the question
    doc = nlp(question)
    
    # Extract named entities (for example, to understand which key concepts are mentioned in the question)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    
    # Print extracted entities for reference (optional)
    print(f"Extracted Entities: {entities}")
    
    # Modify or preprocess the question (optional) based on spaCy's analysis
    # Here, we're simply passing the raw question to the model
    response = llm(question)
    
    return response

# Example question
response = process_and_ask("Explain quantum mechanics in simple terms.")
print(response)


Device set to use mps:0
/var/folders/4q/p59t2ccd40zgg62r0lz1s6sc0000gn/T/ipykernel_14026/3197181194.py:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator)
/var/folders/4q/p59t2ccd40zgg62r0lz1s6sc0000gn/T/ipykernel_14026/3197181194.py:33: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(question)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Extracted Entities: []
Explain quantum mechanics in simple terms.

I am a theoretical physicist and currently working on a class project, which is essentially the explanation of quantum physics using simple words. I have a problem with my description and would like to know some of the answers.

What is the proper way to describe the classical physics of the system?

What you should do in your post is:
Use classical physics and explain it to non-physics students. They tend to get confused by the words they use.

Use the words you use when explaining simple quantum systems. It is not necessary that you explain each element of the system. The quantum system is not a system in the classical level! It should not be at a classical level. The basic physics terms should be explained. These would include:

1. The properties of the system, such as density (quantum) and momentum.
2. The position in space and the velocity in time.
3. The observables (which are determined by the position in space a

In [10]:
import spacy
import random

# Load NLP model for better entity recognition
nlp = spacy.load("en_core_web_sm")

# Sample text data for entity recognition
sample_texts = [
    "Apple is looking at buying U.K. startup for $1 billion",
    "Microsoft plans to launch a new product next year.",
    "Elon Musk is the CEO of SpaceX and Tesla.",
    "The Eiffel Tower is located in Paris, France.",
    "Bill Gates is a famous philanthropist and co-founder of Microsoft."
]

# Function to extract entities using spaCy
def extract_entities(text):
    # Process the text using spaCy's NLP pipeline
    doc = nlp(text)
    # Extract and return named entities as a list of (entity, label) tuples
    return [(ent.text, ent.label_) for ent in doc.ents]

# Pick a random text from the sample list
random_text = random.choice(sample_texts)

# Extract and print entities from the random text
print(f"Text: {random_text}")
entities = extract_entities(random_text)
print(f"Entities: {entities}")


Text: Microsoft plans to launch a new product next year.
Entities: [('Microsoft', 'ORG'), ('next year', 'DATE')]
